This experiment notebook is the base for the lake_to_dwh.py file. It will read/load the newspaper htmls, transform them and prepare the features, which are needed for an easy analyse. The analyse will be about the word "klima" and its changing use over time by newspaper. See Readme for more.

In [1]:
import os
import sys
sys.path.append(os.path.abspath("pylib"))

import pandas as pd
import handle_sqlite

# Load all the newspapers
Here we will load the csv for one day.

load sqlite if able

get all paths for all newspapers (extend with optional daterange (otherwise all))

In [1]:
def run_extract(paper):
    return pd.Series(['a' + paper, 'b' + paper])

for every collected path open and process the newspaper and append result to list

take list and append to sqlite.

In [7]:
# For every day
    # for every newspaper
        # run extract
PATH = ""

df = pd.Series(['c', 'd'])

days = ['Mo','Di','Mi']
papers = ['TAZ','SZ','RBB']


for day in days:
    for paper in papers:
        df= pd.concat([df, run_extract(paper)], ignore_index=True)

#table_into_dwh(df)
df

0        c
1        d
2     aTAZ
3     bTAZ
4      aSZ
5      bSZ
6     aRBB
7     bRBB
8     aTAZ
9     bTAZ
10     aSZ
11     bSZ
12    aRBB
13    bRBB
14    aTAZ
15    bTAZ
16     aSZ
17     bSZ
18    aRBB
19    bRBB
dtype: object